# Time travel (时间旅行)


In [1]:
from typing import Annotated
from dotenv import load_dotenv

from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from typing_extensions import TypedDict
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

load_dotenv()


True

In [2]:
llm = init_chat_model('deepseek:deepseek-chat')


class State(TypedDict):
  messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
  return {'messages': [llm_with_tools.invoke(state['messages'])]}


graph_builder.add_node('chatbot', chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node('tools', tool_node)

graph_builder.add_conditional_edges(
  'chatbot',
  tools_condition,
)
graph_builder.add_edge('tools', 'chatbot')
graph_builder.add_edge(START, 'chatbot')
graph_builder.add_edge('chatbot', END)

memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)

## Add steps

向图表中添加步骤。每个步骤将在其状态历史记录中进行检查点记录：


In [3]:
config = {'configurable': {'thread_id': '1'}}
events = graph.stream(
  {
    'messages': [
      {
        'role': 'user',
        # 提示可以调用工具，不然 deepseek 自主调用工具的意愿很低
        'content': ('我正在学习 LangGraph。你能帮我做一些研究吗？你可以调用 TavilySearch 工具。'),
      },
    ],
  },
  config,
  stream_mode='values',
)
for event in events:
  if 'messages' in event:
    event['messages'][-1].pretty_print()

================================ Human Message =================================

我正在学习 LangGraph。你能帮我做一些研究吗？你可以调用 TavilySearch 工具。
================================== Ai Message ==================================

当然可以！LangGraph 是一个用于构建和操作图结构的库，通常用于自然语言处理（NLP）和机器学习任务。我可以帮你搜索相关的资料、教程、文档或最新动态。

请告诉我你具体想了解 LangGraph 的哪些方面？例如：
1. 基础教程或入门指南
2. 高级功能或应用案例
3. 与其他工具（如 LangChain）的集成
4. 最新版本或更新内容
5. 其他特定问题

或者你可以直接告诉我你的学习目标，我会更有针对性地帮你搜索。
Tool Calls:
  tavily_search (call_0_7e5c67ec-7377-4563-a190-392ba30c79b5)
 Call ID: call_0_7e5c67ec-7377-4563-a190-392ba30c79b5
  Args:
    query: LangGraph tutorial
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph tutorial", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.datacamp.com/tutorial/langgraph-tutorial", "title": "LangGraph Tutorial: What Is LangGraph and How to Use It?", "content": "Imagine you're building a 

In [4]:
events = graph.stream(
  {
    'messages': [
      {
        'role': 'user',
        'content': ('是的，这很有帮助。也许我可以用它来构建一个自主智能体！你可以调用 TavilySearch 工具。'),
      },
    ],
  },
  config,
  stream_mode='values',
)
for event in events:
  if 'messages' in event:
    event['messages'][-1].pretty_print()

================================ Human Message =================================

是的，这很有帮助。也许我可以用它来构建一个自主智能体！你可以调用 TavilySearch 工具。
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_0_da1e55ac-5180-44af-b06b-f01558951e93)
 Call ID: call_0_da1e55ac-5180-44af-b06b-f01558951e93
  Args:
    query: LangGraph autonomous agent
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph autonomous agent", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.langchain.com/langgraph", "title": "LangGraph - LangChain", "content": "Design agent-driven user experiences with LangGraph Platform's APIs. Quickly deploy and scale your application with infrastructure built for agents. LangGraph sets the foundation for how we can build and scale AI workloads — from conversational agents, complex task automation, to

## Replay the full state history

现在已经向聊天机器人添加了步骤，可以 `replay` 查看完整的状态历史记录，以查看所有发生的内容。


In [5]:
to_replay = None
for state in graph.get_state_history(config):
  print('消息数量: ', len(state.values['messages']), '下一个节点: ', state.next)
  print('-' * 80)
  if len(state.values['messages']) == 6:
    # 我们根据消息数量选择一个特定的状态。
    to_replay = state

消息数量:  8 下一个节点:  ()
--------------------------------------------------------------------------------
消息数量:  7 下一个节点:  ('chatbot',)
--------------------------------------------------------------------------------
消息数量:  6 下一个节点:  ('tools',)
--------------------------------------------------------------------------------
消息数量:  5 下一个节点:  ('chatbot',)
--------------------------------------------------------------------------------
消息数量:  4 下一个节点:  ('__start__',)
--------------------------------------------------------------------------------
消息数量:  4 下一个节点:  ()
--------------------------------------------------------------------------------
消息数量:  3 下一个节点:  ('chatbot',)
--------------------------------------------------------------------------------
消息数量:  2 下一个节点:  ('tools',)
--------------------------------------------------------------------------------
消息数量:  1 下一个节点:  ('chatbot',)
--------------------------------------------------------------------------------
消息数量:  0 下一个节点:  ('__st

检查点会在图的每个步骤中保存。这跨越了方法调用，因此您可以回溯到整个线程的历史记录。

### Resume from a checkpoint


In [6]:
print(to_replay.next)
print(to_replay.config)

('tools',)
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f075f49-96f4-6244-8006-97410e0dae28'}}


## Load a state from a moment-in-time

检查点的 `to_replay.config` 包含一个 `checkpoint_id` 时间戳。提供此 `checkpoint_id` 值会告知 LangGraph 的检查点工具从该时间点加载状态。


In [7]:
# The `checkpoint_id` in the `to_replay.config` corresponds to a state we've persisted to our checkpointer.
# `to_replay.config` 中的 `checkpoint_id` 对应于我们已保存到检查点工具的某个状态。
for event in graph.stream(None, to_replay.config, stream_mode='values'):
  if 'messages' in event:
    event['messages'][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_0_da1e55ac-5180-44af-b06b-f01558951e93)
 Call ID: call_0_da1e55ac-5180-44af-b06b-f01558951e93
  Args:
    query: LangGraph autonomous agent
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph autonomous agent", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.langchain.com/langgraph", "title": "LangGraph - LangChain", "content": "Design agent-driven user experiences with LangGraph Platform's APIs. Quickly deploy and scale your application with infrastructure built for agents. LangGraph sets the foundation for how we can build and scale AI workloads — from conversational agents, complex task automation, to custom LLM-backed experiences that 'just work'. The next chapter in building complex production-ready features with LLMs is agentic

图表从 tools 节点继续执行。您可以通过第一个打印的值是搜索引擎工具的响应来判断这一点。
